# HW1 - part 6
<a id=part6></a>

In [10]:
import sys, random,binascii,AES,jupyter_utils,random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import Markdown, display,HTML
%matplotlib inline 
plt.rcParams['figure.dpi'] = 150


This section will focus on the Diffusion principle and examine the extent to which a change in input or internal states affects the end result. The runs in the strain part will be performed on a single input block (16 characters)

In [11]:
plaintext = "From the Pope to"
print("Total Length:",len(plaintext))
display(Markdown(f'\n"**_{plaintext}_**"'))


Total Length: 16



"**_From the Pope to_**"

In [12]:
def tobits(s):
    result = []
    for c in s:
        bits = bin(ord(c))[2:]
        bits = '00000000'[len(bits):] + bits
        result.extend([int(b) for b in bits])
    return result

def frombits(bits):
    chars = []
    for b in range(len(bits) // 8 ):
        byte = bits[b*8:(b+1)*8]
        chars.append(chr(int(''.join([str(bit) for bit in byte]), 2)))
    return ''.join(chars)

def count_flips_to_convert(a, b):

    diff = a ^ b

    # count number of ones in diff
    count = 0
    while diff:
        diff &= (diff - 1)
        count += 1
    return count

def flip_plaintext(plaintext):
    bitflip_p = tobits(plaintext)
    place = random.choice(range(len(bitflip_p)))
    bitflip_p[place] = int(1 - bitflip_p[place])
    bitflip_plaintext = frombits(bitflip_p)
    return bitflip_plaintext,place
bitflip_plaintext,place = flip_plaintext(plaintext)
print("The bit which was chosen to be replaced: ", place)
print("Result:",bitflip_plaintext)

The bit which was chosen to be replaced:  110
Result: From the Pope"to


And we will try to encrypt the message, using an encryption key **"R is a Short key"**
 

In [13]:
key = "R is a Short key"
print(f"'R is a Short key':\n 0x{key.encode().hex()}")

keys = 7*[key]
place = 7*[""]
aes_lib = 7*[""]

aes_lib[0] = AES.AES(int(keys[0].encode().hex(),base=16))
print("0st key wasn't flipped")
for i in range(6):
    keys[i+1],place[i+1] = flip_plaintext(key)
    print(str(i+1) + "st key was flipped at place " + str(place[i+1]))
    aes_lib[i+1] = AES.AES(int(keys[i+1].encode().hex(),base=16))

'R is a Short key':
 0x5220697320612053686f7274206b6579
0st key wasn't flipped
1st key was flipped at place 76
2st key was flipped at place 58
3st key was flipped at place 31
4st key was flipped at place 55
5st key was flipped at place 34
6st key was flipped at place 101


## Question 6:
1. Run the AES algorithm, and save the intermediate results after each round (including the final result)
2. Randomly select a bit in the key and change it from 0 -> 1 or 1-> 0 accordingly Repeat (1) and calculate how many bits changed their value as a result of the change of the single bit in each round, and the encrypted message (relative to the same situation when no errors were injected)
4. Repeat the experiment 5 or 6  times when each time you change one bit in relation to the message in (1) at the end of the runs, present a table

In [14]:
def run_aes_stages(plaintext, key_num):
    spec_enc_text = [""]*10
    for i in range(0, len(plaintext), 16):  
        ciper = aes_lib[key_num].encrypt_by_stage(int(plaintext[i:i+16].encode().hex(),base=16))
        for j,c in enumerate(ciper):
            spec_enc_text[j] += ('{:x}'.format(c)).zfill(32)
    return spec_enc_text
spec_enc_text = run_aes_stages(plaintext, 0)
display(spec_enc_text)

['cb95e6a2a56a602164f8bdc338622f85',
 'cbf42cb3f27bbb6266e1e7abbc87199e',
 '5953b07e2a304dc27be50e6355176310',
 '203d5627e023761e3d86efd4ace38e38',
 'b874b055f6f6fc733e8c7952dbf7ea9c',
 '1dba47880ed16f3c72d966c2526347cc',
 '737bf4f4317604824733b3b13695b2cf',
 'd0d5082d7e23c65c2e2cd9304833f9f6',
 '54d92855bb1e7c8075c0eb813950e85d',
 '793ec12a27911757630e1fd3c4affd06']

In [15]:
print("Displaying output with another key")
bitflip_spec_enc_text = run_aes_stages(plaintext, 1)
display(bitflip_spec_enc_text)

Displaying output with another key


['cb95e6a28faf8fce64f0bdc3386a2f85',
 '150257c8078f0330a0d4d5ef4c79443d',
 'cb3fe5e533c8a0b4e9b7c10da2cd0230',
 'cb8a3334f040f4dc3bc1ab619e270baa',
 'fe8119a88e15aeb86cf62b573f96ceaf',
 '1b41fa229eec4587291a80e4202ff4ff',
 'b3260284807cf48689f402d1504431f0',
 '63a613f7f63a9f15a3c8d6ba71f3a960',
 'f12adadf880b0bb65bcefaeb36d8eb64',
 'fe8480730fa0e0358e4ab2d29a7f08e3']

In [16]:
def calulate_diffrence(textlist,flip_textlist):
    diff = []
    for i in range(len(flip_textlist)):
        p =  int(textlist[i],base=16)
        c =  int(flip_textlist[i],base=16)
        diff.append(count_flips_to_convert(p,c))
    return diff
diff = calulate_diffrence(bitflip_spec_enc_text,spec_enc_text)
display(diff)

[23, 75, 68, 54, 63, 68, 60, 58, 60, 63]

In [17]:
table_result = np.zeros((7,len(bitflip_spec_enc_text)))

for i in range(6):
    bfp_stage = run_aes_stages(plaintext, key_num=1+i)
    table_result[i] = calulate_diffrence(bfp_stage,spec_enc_text)


In [18]:
table_result[6] = np.mean(table_result[:6], axis = 0)
display(pd.DataFrame(table_result, columns=[f"Round {col+1}" for col in range(len(bitflip_spec_enc_text))]))

,Round 1,Round 2,Round 3,Round 4,Round 5,Round 6,Round 7,Round 8,Round 9,Round 10
0,23.000000,75.000000,68.000000,54.000000,63.0,68.0,60.000000,58.0,60.000000,63.000000
1,13.000000,68.000000,58.000000,58.000000,65.0,54.0,63.000000,61.0,69.000000,59.000000
2,22.000000,62.000000,62.000000,67.000000,66.0,63.0,69.000000,61.0,61.000000,66.000000
3,21.000000,66.000000,65.000000,63.000000,69.0,67.0,66.000000,60.0,65.000000,58.000000
4,19.000000,63.000000,58.000000,71.000000,68.0,60.0,57.000000,64.0,68.000000,61.000000
5,27.000000,60.000000,54.000000,67.000000,56.0,69.0,50.000000,59.0,66.000000,61.000000
6,20.833333,65.666667,60.833333,63.333333,64.5,63.5,60.833333,60.5,64.833333,61.333333
